<a href="https://colab.research.google.com/github/shivam-bhandari/LSTM-Classifier/blob/main/LSTM_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import re
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import one_hot
from sklearn.metrics import confusion_matrix,accuracy_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
df = pd.read_csv('train.csv')
df.dropna(inplace=True)
df.reset_index(inplace=True)
df.head(10)

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
8,10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
9,11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


In [16]:
X = df['title']
y = df['label']

In [ ]:
ps = PorterStemmer()
corpus = []

for i in range(len(X)):
    text = X[i]
    text = re.sub('[^a-zA-Z]',' ',text)
    text = text.lower()
    text = text.split()
    text = [ps.stem(t) for t in text if t not in stopwords.words('english')]
    corpus.append(' '.join(text))

In [ ]:
vocab_size = 5000
sent_len = 20

one_hot_encoded = [one_hot(x,vocab_size) for x in corpus]
one_hot_encoded = pad_sequences(one_hot_encoded,maxlen=sent_len)
one_hot_encoded[0]

In [ ]:
X = np.array(one_hot_encoded)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
no_of_output_features = 40

model = Sequential()

model.add(Embedding(vocab_size,no_of_output_features,input_length=sent_len))
model.add(Dropout(0.5))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

In [12]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=64,epochs=40)

Epoch 1/40
192/192 [==============================] - 12s 47ms/step - loss: 0.5200 - accuracy: 0.7621 - val_loss: 0.2383 - val_accuracy: 0.9062
Epoch 2/40
192/192 [==============================] - 8s 39ms/step - loss: 0.2376 - accuracy: 0.9219 - val_loss: 0.2632 - val_accuracy: 0.9170
Epoch 3/40
192/192 [==============================] - 9s 46ms/step - loss: 0.1930 - accuracy: 0.9362 - val_loss: 0.3199 - val_accuracy: 0.9137
Epoch 4/40
192/192 [==============================] - 11s 55ms/step - loss: 0.1962 - accuracy: 0.9442 - val_loss: 0.4125 - val_accuracy: 0.9138
Epoch 5/40
192/192 [==============================] - 7s 38ms/step - loss: 0.1682 - accuracy: 0.9533 - val_loss: 0.3609 - val_accuracy: 0.8882
Epoch 6/40
192/192 [==============================] - 9s 45ms/step - loss: 0.2111 - accuracy: 0.9259 - val_loss: 0.5223 - val_accuracy: 0.8321
Epoch 7/40
192/192 [==============================] - 9s 45ms/step - loss: 0.1828 - accuracy: 0.9460 - val_loss: 0.5853 - val_accuracy: 0.89

In [13]:
pred = model.predict_classes(X_test)
confusion_matrix(y_test,pred)

AttributeError: ignored

In [ ]:
accuracy_score(y_test,pred)